In [4]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from meeko import MoleculePreparation, PDBQTWriterLegacy
import shutil
import glob

def clear_output_directory(output_dir):
    """ Удаляет все файлы в указанной директории. """
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

def read_sdf_file(sdf_file):
    """ Чтение молекул из SDF файла. """
    suppl = Chem.SDMolSupplier(sdf_file)
    molecules = []
    for i, mol in enumerate(suppl):
        if mol is None:
            print(f"Ошибка чтения SDF: запись {i} повреждена или неверная")
        else:
            molecules.append(mol)
    print(f"Прочитано молекул из SDF: {len(molecules)}")
    return molecules

def read_smiles_file(smiles_file):
    """ Чтение молекул из SMILES файла с их названиями, игнорируя строку заголовка. """
    with open(smiles_file, 'r') as f:
        lines = f.readlines()

    molecules = []
    for i, line in enumerate(lines):
        if i == 0 and line.lower().startswith("smiles name"):
            continue 

        parts = line.strip().split()
        if len(parts) >= 2:
            smile, name = parts[0], parts[1]
            mol = Chem.MolFromSmiles(smile)
            if mol:
                mol.SetProp('_Name', name)
                molecules.append(mol)
            else:
                print(f"Ошибка чтения SMILES: строка {i}, {smile}")
    print(f"Прочитано молекул из SMILES: {len(molecules)}")
    return molecules



def prepare_molecule(mol):
    """ Добавление атомов водорода и попытка генерации 3D структуры. """
    try:
        mol = Chem.AddHs(mol)
        if AllChem.EmbedMolecule(mol, AllChem.ETKDG()) == -1:  # Проверка на успешную вставку
            print(f"Не удалось сгенерировать 3D структуру для молекулы {mol.GetProp('_Name')}")
            return None
        AllChem.MMFFOptimizeMolecule(mol)  # Оптимизация с использованием MMFF
        return mol
    except Exception as e:
        print(f"Ошибка при подготовке молекулы {mol.GetProp('_Name')}: {e}")
        return None

def convert_to_pdbqt(mol, output_dir, mol_name):
    """ Конвертация молекулы в PDBQT и сохранение в файл. """
    try:
        prep = MoleculePreparation(rigid_macrocycles=False, min_ring_size=6)
        prep.prepare(mol)
        writer = PDBQTWriterLegacy()
        pdbqt_content = writer.write_string(prep.setup)[0]
        with open(os.path.join(output_dir, f"{mol_name}.pdbqt"), 'w') as file:
            file.write(pdbqt_content)
        return True
    except Exception as e:
        print(f"Ошибка при конвертации в PDBQT: {mol_name}, {e}")
        return False

def process_directory(input_dir, output_dir):
    """ Обработка молекул из SDF и SMILES файлов в директории и конвертация в PDBQT. """
    clear_output_directory(output_dir)  # Очистка директории перед обработкой

    successful_conversions = 0
    for sdf_file in glob.glob(os.path.join(input_dir, '*.sdf')):
        sdf_output_dir = os.path.join(output_dir, "sdf")
        os.makedirs(sdf_output_dir, exist_ok=True)
        for mol in read_sdf_file(sdf_file):
            if mol is not None:
                mol = prepare_molecule(mol)
                if mol:
                    mol_name = mol.GetProp('_Name') if mol.HasProp('_Name') else 'Unnamed'
                    if convert_to_pdbqt(mol, sdf_output_dir, mol_name):
                        successful_conversions += 1

    for smiles_file in glob.glob(os.path.join(input_dir, '*.smi')):
        smiles_output_dir = os.path.join(output_dir, "smiles")
        os.makedirs(smiles_output_dir, exist_ok=True)
        for mol in read_smiles_file(smiles_file):
            if mol is not None:
                mol = prepare_molecule(mol)
                if mol:
                    mol_name = mol.GetProp('_Name') if mol.HasProp('_Name') else 'Unnamed'
                    if convert_to_pdbqt(mol, smiles_output_dir, mol_name):
                        successful_conversions += 1

    print(f"Успешно конвертировано в PDBQT: {successful_conversions} молекул")

# Путь к директории с файлами и директория для сохранения результатов
input_dir = './vina_data'
output_dir = './pdbqt_output'

process_directory(input_dir, output_dir)


Прочитано молекул из SDF: 30
Прочитано молекул из SMILES: 30


[17:02:09] UFFTYPER: Unrecognized charge state for atom: 27


Успешно конвертировано в PDBQT: 60 молекул


Код ниже ловил ошибку
[23:45:24] UFFTYPER: Unrecognized charge state for atom: 27
она возникает у CHEMBL4908646

Как я понимаю проблема в атоме фтора, для которого силовое поле не подходит


In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem

def read_smiles_file(smiles_file):
    """Чтение SMILES из файла с их названиями."""
    with open(smiles_file, 'r') as file:
        lines = file.readlines()

    smiles_data = []
    for i, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) >= 2:
            smile, name = parts[0], parts[1]
            smiles_data.append((smile, name))
        else:
            print(f"Неверный формат строки {i}: {line.strip()}")
    return smiles_data

def find_problematic_smiles(smiles_data):
    """Поиск SMILES, вызывающих ошибки при создании молекулы."""
    for smile, name in smiles_data:
        print(f"Обработка SMILES: {smile} (имя: {name})")  # Вывод информации о текущей молекуле
        try:
            mol = Chem.MolFromSmiles(smile)
            if mol is None:
                raise ValueError("Не удалось создать молекулу")
            mol = Chem.AddHs(mol)
            if AllChem.EmbedMolecule(mol, AllChem.ETKDG()) == -1:
                raise ValueError("Не удалось сгенерировать 3D структуру")
        except Exception as e:
            print(f"Ошибка для SMILES '{smile}' (имя: {name}): {e}")

# Замените путь к вашему SMILES файлу здесь
smiles_file = './vina_data/example.smi'
smiles_data = read_smiles_file(smiles_file)
find_problematic_smiles(smiles_data)


Неверный формат строки 0: 
Обработка SMILES: CC(=O)N1C[C@@H]2OCC(=O)N(Cc3ccc(F)cc3)[C@@H]2C1 (имя: CHEMBL4959907)
Обработка SMILES: Cc1nn(C)cc1-c1cc(C(N)=O)nc2cc(CN3C[C@@H](C)O[C@@H](C(F)(F)F)C3)ccc12 (имя: CHEMBL4108673)
Обработка SMILES: Cc1nsc(-c2cc(C(N)=O)nc3cc(CN4C[C@@H](C)O[C@@H](C(F)(F)F)C4)ccc23)n1 (имя: CHEMBL4107154)
Обработка SMILES: Fc1ccc(CN2CCOC(Cn3cncn3)C2)c2ncccc12 (имя: CHEMBL3460453)
Обработка SMILES: COc1ccc(Cc2nc3ccc(C(=O)N4C[C@@H](C)O[C@@H](C)C4)cc3o2)cc1OC (имя: CHEMBL1539631)
Обработка SMILES: CCOc1ccc(NS(=O)(=O)c2cc(C(=O)N3CC(C)OC(C)C3)ccc2C)cc1 (имя: CHEMBL1864849)
Обработка SMILES: Cc1ccc(NS(=O)(=O)c2ccc(C(=O)N3CC(C)OC(C)C3)cc2)cc1 (имя: CHEMBL1385066)
Обработка SMILES: COc1cc(/C=C2\OC(C)(C)CN([C@H](c3cc(F)c(F)c(F)c3)[C@@H](C)O)C2=O)ccc1-n1cnc(C)c1 (имя: CHEMBL2151107)
Обработка SMILES: COCC1CN(Cc2ccc3c(-c4cnn(C)c4)cc(C(N)=O)nc3c2)CCO1 (имя: CHEMBL3979211)
Обработка SMILES: COc1cc(N)c(Cl)cc1C(=O)NCC1CN(Cc2ccccc2)CCO1 (имя: CHEMBL30982)
Обработка SMILES: O=C(O)

[16:46:20] UFFTYPER: Unrecognized charge state for atom: 27


Обработка SMILES: C[C@]12CC[C@@]3(C[C@@H]1CC[C@@H]1[C@@H]2CC[C@]2(C)C(=O)CC[C@@H]12)CN(Cc1ccccc1)[C@@H](Cc1ccccc1)C(=O)O3 (имя: CHEMBL3088089)
Обработка SMILES: CC1OC2(CCN(CCc3ccccc3)CC2)CN(Cc2ccccc2)C1=O (имя: CHEMBL4748898)
Обработка SMILES: O=C(/C=C/c1cncc(F)c1-c1cnn(C2CC2)c1)Nc1ccc(CN2CC3CC(C2)O3)cc1 (имя: CHEMBL4783219)
Обработка SMILES: C[C@H]1N(C(=O)c2ccc(C(F)(F)F)cc2)CC(=O)O[C@@]1(Cn1cncn1)c1ccc(C(F)(F)F)cc1 (имя: CHEMBL126507)
Обработка SMILES: COc1cc(CN2CC(C)OC(C)C2)cc2[nH]c(=O)c3c(c12)NCCC3 (имя: CHEMBL3649793)
Обработка SMILES: N#Cc1ccc(CN2CC3CN(CCNS(=O)(=O)c4ccc(F)cc4)CC(C2)O3)cc1 (имя: CHEMBL3957375)
